# 📊 인터랙티브 데이터 필터링 예제
## 실제 사용 시나리오별 필터링 방법

이 노트북에서는 실제 이메일 마케팅에서 자주 사용되는 필터링 시나리오들을 다룹니다.

In [ ]:
# 라이브러리 import 및 설정
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import re
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (Mac용)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 로드
data_path = '/Users/milo/Desktop/ocean/영중소구간필터링/202508최종취합/서울성남_통신판매사업자_완전통합.csv'
df = pd.read_csv(data_path)

print(f"✅ 데이터 로드 완료: {len(df):,}개 레코드")
print(f"📊 컬럼: {list(df.columns)}")

## 🎯 시나리오 1: 이메일 마케팅 가능한 타겟 추출
유효한 이메일을 가진 업체들만 필터링

In [ ]:
# 유효한 이메일을 가진 업체 필터링
def get_valid_email_targets(df):
    """
    이메일 마케팅이 가능한 타겟들을 추출
    - 이메일이 존재하고
    - 마스킹되지 않은 완전한 이메일
    - 정상영업 상태
    """
    filtered = df[
        # 이메일이 존재하고
        (df['전자우편'].notna()) & 
        (df['전자우편'] != '') &
        # 마스킹되지 않은 완전한 이메일 (별표 없음)
        (~df['전자우편'].str.contains('\*', na=False)) &
        # 정상영업 상태
        (df['업소상태'] == '정상영업')
    ].copy()
    
    return filtered

# 실행
email_targets = get_valid_email_targets(df)
print(f"🎯 이메일 마케팅 가능 타겟: {len(email_targets):,}개 ({len(email_targets)/len(df)*100:.1f}%)")

# 상위 10개 미리보기
print("\n📋 상위 10개 타겟:")
email_targets[['상호', '대표자명', '전자우편', '법인구분']].head(10)

## 🏢 시나리오 2: 법인 vs 개인사업자 분석

In [ ]:
# 법인구분별 분석
business_type_analysis = email_targets['법인구분'].value_counts()
print("📊 법인구분별 분포:")
for btype, count in business_type_analysis.items():
    percentage = count / len(email_targets) * 100
    print(f"  {btype}: {count:,}개 ({percentage:.1f}%)")

# 시각화
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
business_type_analysis.plot(kind='pie', autopct='%1.1f%%')
plt.title('법인구분별 분포')

plt.subplot(1, 2, 2)
business_type_analysis.plot(kind='bar')
plt.title('법인구분별 개수')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 🌐 시나리오 3: 온라인 플랫폼별 분류
네이버, 쿠팡, 자체 웹사이트 등으로 분류

In [ ]:
def categorize_platform(domain):
    """
    인터넷도메인을 기반으로 플랫폼 분류
    """
    if pd.isna(domain) or domain == '':
        return '정보없음'
    
    domain_lower = str(domain).lower()
    
    if 'naver' in domain_lower or '네이버' in domain_lower or 'smartstore' in domain_lower:
        return '네이버'
    elif 'coupang' in domain_lower or '쿠팡' in domain_lower:
        return '쿠팡'
    elif 'gmarket' in domain_lower or 'g마켓' in domain_lower:
        return 'G마켓'
    elif 'auction' in domain_lower or '옥션' in domain_lower:
        return '옥션'
    elif 'toss' in domain_lower:
        return '토스페이먼츠'
    elif any(platform in domain_lower for platform in ['http', 'www', '.com', '.co.kr']):
        return '자체웹사이트'
    else:
        return '기타플랫폼'

# 플랫폼 분류 적용
email_targets['플랫폼분류'] = email_targets['인터넷도메인'].apply(categorize_platform)

# 플랫폼별 분석
platform_analysis = email_targets['플랫폼분류'].value_counts()
print("🌐 플랫폼별 분포:")
for platform, count in platform_analysis.items():
    percentage = count / len(email_targets) * 100
    print(f"  {platform}: {count:,}개 ({percentage:.1f}%)")

# 시각화
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
platform_analysis.plot(kind='bar')
plt.title('플랫폼별 업체 수')
plt.xticks(rotation=45)

plt.subplot(2, 2, 2)
platform_analysis.plot(kind='pie', autopct='%1.1f%%')
plt.title('플랫폼별 비율')

# 법인구분 x 플랫폼 크로스탭
plt.subplot(2, 2, 3)
crosstab = pd.crosstab(email_targets['법인구분'], email_targets['플랫폼분류'])
sns.heatmap(crosstab, annot=True, fmt='d', cmap='Blues')
plt.title('법인구분 x 플랫폼 분포')

plt.tight_layout()
plt.show()

print("\n📊 법인구분 x 플랫폼 크로스탭:")
print(crosstab)